<a href="https://colab.research.google.com/github/arka57/Hindi-Vowel-Consonant-Classification/blob/main/Hindi_Vowel_Consonant_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing libraries

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
cd MyDrive/DL/project/Hindi_Vowel_Consonant

/gdrive/MyDrive/DL/project/Hindi_Vowel_Consonant


In [ ]:
ls

sample_data/


In [ ]:
!unzip train.zip

In [ ]:
!unzip test.zip

In [ ]:
ls

test/  test.zip  train/  train.zip


In [ ]:
#Code for making the image data in tensor form

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torchvision
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms

#For converting the dataset to torchvision dataset format
class VowelConsonantDataset(Dataset):
    def __init__(self, file_path,train=True,transform=None):
        self.transform = transform
        self.file_path=file_path
        self.train=train
        self.file_names=[file for _,_,files in os.walk(self.file_path) for file in files]
        self.len = len(self.file_names)
        if self.train:
            self.classes_mapping=self.get_classes()
    def __len__(self):
        return len(self.file_names)
    
    def __getitem__(self, index):
        file_name=self.file_names[index]
        image_data=self.pil_loader(self.file_path+"/"+file_name)
        if self.transform:
            image_data = self.transform(image_data)
        if self.train:
            file_name_splitted=file_name.split("_")
            Y1 = self.classes_mapping[file_name_splitted[0]]
            Y2 = self.classes_mapping[file_name_splitted[1]]
            z1,z2=torch.zeros(10),torch.zeros(10)
            z1[Y1-10],z2[Y2]=1,1
            label=torch.stack([z1,z2])

            return image_data, label

        else:
            return image_data, file_name
          
    def pil_loader(self,path):
        with open(path, 'rb') as f:
            img = Image.open(f)
            return img.convert('RGB')

      
    def get_classes(self):
        classes=[]
        for name in self.file_names:
            name_splitted=name.split("_")
            classes.extend([name_splitted[0],name_splitted[1]])
        classes=list(set(classes))
        classes_mapping={}
        for i,cl in enumerate(sorted(classes)):
            classes_mapping[cl]=i
        return classes_mapping

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import matplotlib.pyplot as plt
from torchvision import datasets

import torchvision.transforms as transforms

import numpy as np
import pandas as pd

train_on_gpu = torch.cuda.is_available()

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor()])

In [ ]:
#Splitting of data into train,validation and test set

In [ ]:
full_data = VowelConsonantDataset("./train",train=True,transform=transform)
train_size = int(0.9 * len(full_data))
test_size = len(full_data) - train_size

train_data, validation_data = random_split(full_data, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=60, shuffle=True)


In [ ]:
test_data = VowelConsonantDataset("./test",train=False,transform=transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=4,shuffle=False)

In [ ]:
type(train_data)

torch.utils.data.dataset.Subset

In [ ]:
dataiter=iter(train_loader)
images,labels=dataiter.next()
print(images.shape)
print(images[0].shape)

torch.Size([4, 3, 64, 64])
torch.Size([3, 64, 64])


In [ ]:
#image dimesnion: 3X64X64

In [ ]:
#First Model Applied--LeNet

In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet,self).__init__()
        self.cnn_model=nn.Sequential(
            nn.Conv2d(3,6,(5,5)),               #(3,64,64)--->(6,60,60)
            nn.AvgPool2d(2,stride=2),           #(6,60,60)--->(6,30,30)
            nn.ReLU(),
            nn.Conv2d(6,16,(5,5)),              #(6,30,30)--->(16,26,26)
            nn.AvgPool2d(2,stride=2),           #(16,26,26)--->(16,13,13)
            nn.ReLU()

        )
        self.vowel_model=nn.Sequential(         #Vowel part--fully connected layers
            nn.Linear(2704,256),
            nn.ReLU(),
            nn.Linear(256,120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,10)
        )    
        self.consonant_model=nn.Sequential(     #Consonant part--fully connected layers
            nn.Linear(2704,256),
            nn.ReLU(),
            nn.Linear(256,120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,10)
        )  

    def forward(self,x):
        x=self.cnn_model(x)
        x=x.view(x.size(0),-1)
        v1=self.vowel_model(x)
        c1=self.consonant_model(x)
        return v1,c1                        #returning vowel,consonant




        

In [ ]:
#Learning Algo


for j,data in enumerate(train_loader,0):
    if(j==1):
        images,labels=data
        break    
print(data[0])    

In [ ]:
print(data[0].shape)

torch.Size([60, 3, 64, 64])


In [ ]:
print(images[0].shape)
print(labels[0].shape)

torch.Size([3, 64, 64])
torch.Size([2, 10])


In [ ]:
for _,i in enumerate(labels,0):
    vowel,consonant=i

In [ ]:
print(vowel.shape)
print(consonant.shape)

torch.Size([10])
torch.Size([10])


In [ ]:
print(vowel)

tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])


In [ ]:
#instantiating network

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
net=LeNet().to(device)
opt=optim.Adam(net.parameters(),weight_decay=0.01)
loss_fn=nn.CrossEntropyLoss()

In [ ]:
#Evaluation Function

In [ ]:
def evaluation(dataloader):
    total=0
    correct=0
    for i in dataloader:
        images,labels=i
        images,labels=images.to(device),labels.to(device)
        for j in labels:
            vowel_label,consonant_label=j
        vowel,consonant=net.forward(images)
        _,vowel_pred=torch.max(vowel,1)
        _,consonant_pred=torch.max(consonant,1)
        total+=labels.size(0)
        correct+=((vowel_pred==vowel_label) and (consonant_pred==consonant_label)).sum().item()
    return (correct/total)*100

In [ ]:
#Learning Algo

In [ ]:
loss_epoch=[]
epochs=3

for i in range(epochs):
    for _,data in enumerate(train_loader,0):
        input,labels=data
        input,labels=input.to(device),labels.to(device)
        #print(input.shape)
        #print(labels.shape)
        
        
        vowel_label,consonant_label=labels[:,0,:],labels[:,1,:]
        
        vowel,consonant=net.forward(input)
        #print(vowel_label.shape)
        #print(consonant_label.shape)
        #print(vowel.shape)
        #print(consonant.shape)
        lossV=loss_fn(vowel,vowel_label)
        lossC=loss_fn(consonant,consonant_label)
        loss=lossV+lossC
        loss.backward()

        opt.step()
        opt.zero_grad()

        del vowel,consonant,labels,input
        torch.cuda.empty_cache()
        
    loss_epoch.append(loss.item())
    print('Epoch %d/%d, Training Accuaracy 0.2%f, Test Accuracy 0.2%f' % (i,epochs,evaluate(train_loader),evaluate(test_loader))) 

plt.plot(loss_epoch)
plt.show()           


NameError: ignored